In [ ]:
4

N09E105 /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem__Fw.tif /media/ljp238/12TBWolf/BRCHIEVE/TDEM12VFILL/N09E105/N09E105_TDX12_WGS.tif
Processando: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem__Fw.tif -> mlinterps
read_dem
mask_invalid_values
Training new model: catboost
interpolate_missing_values
No valid data to train the model or no missing values to interpolate.


ValueError: too many values to unpack (expected 2)

N09E105 /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem__Fw.tif /media/ljp238/12TBWolf/BRCHIEVE/TDEM12VFILL/N09E105/N09E105_TDX12_WGS.tif
Processando: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem__Fw.tif -> mlinterps


NameError: name 'mlinterpe' is not defined

In [ ]:
string = 
tilename = string[-2]
tilename

'N09E105'

In [67]:
import os
import time
import numpy as np
from glob import glob
import rasterio
from catboost import CatBoostRegressor
import joblib  # For saving and loading models
import json    # For saving parameters


def get_best_gpu():
    """Selects the best available GPU based on memory size."""
    try:
        import torch
        if not torch.cuda.is_available():
            return 'CPU'
        
        num_gpus = torch.cuda.device_count()
        if num_gpus == 1:
            return 'cuda:0'
        
        best_gpu = max(range(num_gpus), key=lambda i: torch.cuda.get_device_properties(i).total_memory)
        return f'cuda:{best_gpu}'
    except ImportError:
        return 'CPU'


def read_dem(dem_path):
    """Reads a DEM file and returns the data as a NumPy array along with its metadata."""
    with rasterio.open(dem_path) as src:
        data = src.read(1).astype(np.float32)  # Read first band
        meta = src.meta
    return data, meta


def mask_invalid_values(data, nodata_value=-9999, min_valid=-50, max_valid=2000):
    """Masks out invalid values by setting them to NaN."""
    print(f"Masking invalid values: nodata={nodata_value}, min_valid={min_valid}, max_valid={max_valid}")
    data = np.where((data <= min_valid) | (data >= max_valid) | (data == nodata_value), np.nan, data)
    return data


def save_dem(dem_path, data, meta):
    """Saves the processed DEM back to a file."""
    meta.update(dtype=rasterio.float32, nodata=np.nan)
    with rasterio.open(dem_path, 'w', **meta) as dst:
        dst.write(data, 1)


def save_model_and_params(model, model_type, params, dem_opath):
    """Saves the trained model and its parameters to files."""
    model_filename = f"{dem_opath}_{model_type}_model.joblib"
    params_filename = f"{dem_opath}_{model_type}_params.json"
    
    joblib.dump(model, model_filename)
    with open(params_filename, 'w') as param_file:
        json.dump(params, param_file)
    print(f"Model and parameters saved: {model_filename}, {params_filename}")


def interpolate_missing_values_smodel(data, model_type='catboost', model=None, model_params=None, dem_opath=None, meta=None):
    """Performs interpolation on missing values in a DEM using CatBoost."""
    mask = np.isfinite(data)
    coords = np.array(np.nonzero(mask)).T
    values = data[mask]
    missing_coords = np.array(np.nonzero(~mask)).T
    
    print(f"Valid data points: {len(values)}, Missing data points: {len(missing_coords)}")
    
    if len(values) == 0:
        print("No valid data points to train the model.")
        return data, None  # Ensure both data and model are returned
    if len(missing_coords) == 0:
        print("No missing data points to interpolate.")
        return data, None  # Ensure both data and model are returned

    if model is None:
        # Initialize CatBoost model with provided or default parameters
        model_params = model_params or {
            'iterations': 1000,
            'verbose': 200,
            'early_stopping_rounds': 500,
            'task_type': 'GPU' if 'cuda' in get_best_gpu() else 'CPU',
        }
        model = CatBoostRegressor(**model_params)
        model.fit(coords, values)
    
    # Interpolate missing values
    data[~mask] = model.predict(missing_coords)

    # Save training data and mask as GeoTIFF
    if dem_opath and meta:
        # Save training data (coordinates used in the model)
        training_data_filepath = dem_opath.replace('.tif', '_training_data.tif')
        null_mask_filepath = dem_opath.replace('.tif', '_null_mask.tif')

        # Create an empty array to store training data
        training_data = np.full(data.shape, np.nan, dtype=np.float32)
        training_data[tuple(coords.T)] = 1  # Mark training data locations
        
        # Save the training data as a GeoTIFF
        with rasterio.open(training_data_filepath, 'w', **meta) as dst:
            dst.write(training_data, 1)

        # Create an empty array for the null mask
        null_mask = np.full(data.shape, 0, dtype=np.uint8)  # 0 means valid, 1 means missing
        null_mask[~mask] = 1  # Mark missing data as 1

        # Save the null mask as a GeoTIFF
        with rasterio.open(null_mask_filepath, 'w', **meta) as dst:
            dst.write(null_mask, 1)

        print(f"Training data and null mask saved as GeoTIFFs: {training_data_filepath}, {null_mask_filepath}")
    
    return data, model


def mlinterps(dem_ipath, dem_opath, model_type='catboost', model_params=None):
    """Full pipeline: Read, mask, interpolate, save the DEM, and save the model and parameters."""
    print('Reading DEM...')
    data, meta = read_dem(dem_ipath)
    print('Masking invalid values...')
    data = mask_invalid_values(data)
    
    # Load existing model if it exists
    model_filename = f"{dem_opath}_{model_type}_model.joblib"
    params_filename = f"{dem_opath}_{model_type}_params.json"
    model = None
    if os.path.isfile(model_filename):
        print(f"Loading existing model: {model_filename}")
        model = joblib.load(model_filename)
    else:
        print(f"Training new model: {model_type}")
    
    # Interpolate missing values
    print('Interpolating missing values...')
    data, model = interpolate_missing_values_smodel(data, model_type, model, model_params, dem_opath, meta)
    
    print('Saving DEM...')
    save_dem(dem_opath, data, meta)
    
    # Save model and parameters
    params = {'model_type': model_type, 'model_params': model_params}
    save_model_and_params(model, model_type, params, dem_opath)
    print('Pipeline completed.')


def demvfill_unsupervised(dem_ipath, dem_opath, method='mlinterps', smoothing_iterations=0, model_params=None):
    """
    Fills gaps in DEMs using interpolation methods.
    """
    ti = time.perf_counter()

    # Check if the input file exists
    if not os.path.isfile(dem_ipath):
        print(f"Input file not found: {dem_ipath}")
        return
    
    print(f"Processing: {dem_ipath} -> {method}")

    # Mapping of methods to their corresponding functions
    method_map = {
        'mlinterps': (mlinterps, '_mlinterps.tif'),
    }

    # Check if the method is valid
    if method not in method_map:
        print(f"Method not available: {method}. Try: mlinterps")
        return

    # Get the function and output file suffix based on the chosen method
    func, suffix = method_map[method]

    # Process the DEM with the given parameters
    func(dem_ipath, dem_opath, model_type='catboost', model_params=model_params)

    print(f"Completed: {dem_opath}")
    print(f"Execution time: {(time.perf_counter() - ti)/60:.2f} min(s)")


# # Main execution block
# if __name__ == "__main__":
#     BRCHIEVE = "/media/ljp238/12TBWolf/BRCHIEVE"
#     X = 12
#     si = 0
#     method = 'mlinterps'
#     model_params = {
#         'iterations': 1000,
#         'verbose': 200,
#         'early_stopping_rounds': 500,
#         'task_type': 'GPU' if 'cuda' in get_best_gpu() else 'CPU',
#     }

#     tiles_xdpath = f"{BRCHIEVE}/TILES{X}"
#     outdir = f"{BRCHIEVE}/TDEM{X}VFILL"
#     os.makedirs(tiles_xdpath, exist_ok=True)
#     os.makedirs(outdir, exist_ok=True)

#     try:
#         tilenames = os.listdir(tiles_xdpath)
#     except FileNotFoundError:
#         print(f"Directory not found: {tiles_xdpath}")
#         tilenames = []

#     tdem_vfiles = glob(f"{tiles_xdpath}/*/*_tdem_dem__Fw.tif")
#     for i, fi in enumerate(tdem_vfiles):
#         ti = time.perf_counter()
#         if i > 0: break
#         tilename = fi.split('/')[-2]
#         tiles_ydpath = os.path.join(outdir, tilename)
#         os.makedirs(tiles_ydpath, exist_ok=True)
#         fo = f"{tiles_ydpath}/{tilename}_TDX12_WGS.tif"
#         print(f"{tilename} {fi} {fo}")
#         demvfill_unsupervised(fi, fo, method, smoothing_iterations=si, model_params=model_params)
#         print(f"Total execution time: {(time.perf_counter() - ti)/60:.2f} min(s)")
#         print(f"Method: {method} | Tiles processed: {len(tilenames)}")

In [69]:
import os
import time
import numpy as np
from glob import glob
import rasterio
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import pickle

def random_forest_interpolation(data):
    """
    Perform random forest interpolation for missing values in 2D data.
    """
    mask = np.isfinite(data)
    if not np.any(mask):  # Check if there are any finite values
        raise ValueError("Input data contains no finite values for interpolation.")
    
    coords = np.array(np.nonzero(mask)).T
    values = data[mask]
    missing_coords = np.array(np.nonzero(~mask)).T

    # Train a Random Forest Regressor
    model = RandomForestRegressor()
    model.fit(coords, values)

    # Predict missing values
    predicted = model.predict(missing_coords)
    data[~mask] = predicted
    return data, model, coords

def catboost_interpolation(data):
    """
    Perform CatBoost interpolation for missing values in 2D data.
    """
    mask = np.isfinite(data)
    if not np.any(mask):  # Check if there are any finite values
        raise ValueError("Input data contains no finite values for interpolation.")
    
    coords = np.array(np.nonzero(mask)).T
    values = data[mask]
    missing_coords = np.array(np.nonzero(~mask)).T

    # Train a CatBoost Regressor
    model = CatBoostRegressor(verbose=False)
    model.fit(coords, values)

    # Predict missing values
    predicted = model.predict(missing_coords)
    data[~mask] = predicted
    return data, model, coords

def process_tdem_file(input_file, output_file, params):
    """
    Process a single DEM file, perform interpolation, and save the result.
    """
    try:
        with rasterio.open(input_file) as src:
            dem_data = src.read(1, masked=True)  # Read the first band as a NumPy array
            dem_data[dem_data < -30] = np.nan 
            dem_data[dem_data > 100] = np.nan 
            profile = src.profile  # Save metadata for writing output

        # Check if there are any finite values in the data
        if not np.any(np.isfinite(dem_data)):
            raise ValueError(f"No finite values found in {input_file}. Skipping file.")

        # Extract parameters
        method = params["method"]
        smoothing_iterations = params["smoothing_iterations"]
        model_params = params.get("model_params", {})

        # Perform interpolation based on the specified method
        if method == "random_forest":
            interpolated_data, model, coords = random_forest_interpolation(dem_data)
        elif method == "catboost":
            interpolated_data, model, coords = catboost_interpolation(dem_data)
        else:
            raise ValueError(f"Unsupported interpolation method: {method}")

        # Apply smoothing iterations (placeholder logic)
        for _ in range(smoothing_iterations):
            interpolated_data = smooth_data(interpolated_data)

        # Save the interpolated data to a new file
        profile.update(dtype=rasterio.float32)
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(interpolated_data.astype(rasterio.float32), 1)

        # Save the mask as a GeoTIFF
        mask = np.isfinite(dem_data).astype(np.uint8)
        mask_profile = profile.copy()
        mask_profile.update(dtype=rasterio.uint8, count=1)
        mask_file = os.path.splitext(output_file)[0] + "_mask.tif"
        with rasterio.open(mask_file, 'w', **mask_profile) as dst:
            dst.write(mask, 1)

        # Save the model as a pickle file
        model_file = os.path.splitext(output_file)[0] + f"_{method}_model.pkl"
        with open(model_file, 'wb') as f:
            pickle.dump(model, f)

        # Save the training coordinates as a .npy file
        coords_file = os.path.splitext(output_file)[0] + "_training_coords.npy"
        np.save(coords_file, coords)

    except Exception as e:
        print(f"Error processing {input_file}: {e}")

def smooth_data(data):
    """
    Placeholder function for smoothing data.
    Replace with actual smoothing logic if needed.
    """
    return data  # No smoothing applied in this placeholder

if __name__ == "__main__":
    BRCHIEVE = "/media/ljp238/12TBWolf/BRCHIEVE"
    X = 12
    si = 0
    method = 'mlinterps'
    model_params = {
        'iterations': 1000,
        'verbose': 200,
        'early_stopping_rounds': 500,
        'task_type': 'GPU' if 'cuda' in get_best_gpu() else 'CPU',
    }

    tiles_xdpath = f"{BRCHIEVE}/TILES{X}"
    outdir = f"{BRCHIEVE}/TDEM{X}VFILL"
    os.makedirs(tiles_xdpath, exist_ok=True)
    os.makedirs(outdir, exist_ok=True)
    tdem_vfiles = glob(f"{tiles_xdpath}/*/*_tdem_dem__Fw.tif")

    # Define parameters
    params = {
        "method": "catboost",  # Choose between "random_forest" and "catboost"
        "smoothing_iterations": 1,
        "model_params": model_params  # Add model-specific parameters here if needed
    }

    for i, fi in enumerate(tdem_vfiles):
        ti = time.perf_counter()

        # Process only the first file for demonstration purposes
        if i > 0:
            break

        tilename = fi.split('/')[-2]
        tiles_ydpath = os.path.join(outdir, tilename)
        os.makedirs(tiles_ydpath, exist_ok=True)
        fo = f"{tiles_ydpath}/{tilename}_TDX12_WGS.tif"

        print(f"Processing {tilename}: {fi} -> {fo}")
        process_tdem_file(fi, fo, params)
        print(f"Total execution time: {(time.perf_counter() - ti)/60:.2f} min(s)")

Processing N09E105: /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem__Fw.tif -> /media/ljp238/12TBWolf/BRCHIEVE/TDEM12VFILL/N09E105/N09E105_TDX12_WGS.tif
Error processing /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem__Fw.tif: No finite values found in /media/ljp238/12TBWolf/BRCHIEVE/TILES12/N09E105/N09E105_tdem_dem__Fw.tif. Skipping file.
Total execution time: 0.02 min(s)
